## Reduction operation: the sum of the numbers in the range [0, value)

In [13]:
import numpy as np
import multiprocessing as mp
import time 
import sys

def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s


# Secuencial
print ("Secuencial (programa base), con {50000} valores. : \n")

value = int(sys.argv[1])
#value = 5*10**7
X = np.random.rand(value)

# Utilizando bucle for. 
tiempo = %timeit -r 2 -o -q reduc_operation(np.random.rand(5*10**4))
print("Tiempo:", tiempo)
print(f"Resultado de la suma en rango [0, {value}]: {reduc_operation(X)}\n")

print (f"Secuencial usando funciones universales con {value} valores. : \n")

# Utilizando numpy.sum()
tiempo = %timeit -r 2 -o -q np.sum(X)
print("Tiempo usando numpy.sum():", tiempo)
print("Resultado usando numpy.sum():", np.sum(X),"\n ")

# Utilizando numpy.ndarray.sum()
tiempo= %timeit -r 2 -o -q X.sum()
print("Tiempo usando numpy.ndarray.sum():", tiempo)
print("Resultado usando numpy.ndarray.sum():", X.sum())


Secuencial (programa base), con 50000 valores. : 

Tiempo usando numpy.sum(): 69.8 ms ± 801 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
Resultado usando numpy.sum(): 24999299.6682682 
 
Tiempo usando numpy.ndarray.sum(): 68.7 ms ± 12.9 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
Resultado usando numpy.ndarray.sum(): 24999299.6682682


##### a) En paralelo usando "map" de Multiprocessing:

In [2]:


def reduc_operation(A):
    """Compute the sum of the elements of Array A."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s


X = np.random.rand(value) #El argumento iterable de la función MAP, al que se aplicará la función reduc_operation(X)

print (f"En paralelo usando map y Pool de multiprocessing, con {value} valores")



def parallel_sum(array, num_parts):
    """Introduce el array a dividir, y el número de partes en las que fraccionarlo."""
    sub_arrays = np.array_split(array, num_parts)
    
    return sub_arrays

num_partes=[1,2,4]
for num_part in num_partes: #servirá para iterar sobre las partes pedidas por el enunciado del ejercicio. 
    
    # Fraccionamiento del array original
    sub_arrays = np.array_split(X, num_part)

    tiempo_inicial=time.time()
    
    # Defino la pool y el proceso que deseo paralelizar, almacenando los resultados parciales para sumarlos posteriormente.
    with mp.Pool(processes=num_part) as pool:
        suma_parcial = pool.map(reduc_operation, sub_arrays)
        suma_total=sum(suma_parcial)

    tiempo_final=time.time()
    
    print (f"Resultados para un array de tamaño {X.size} dividido en {num_part} partes:")
    print(f"Resultado: {suma_total}") 
    print(f"Tiempo: {tiempo_final - tiempo_inicial:} segundos\n")



Resultados para un array de tamaño 50000000 dividido en 1 partes:
Resultado: 25001185.0508762
Tiempo: 85.90163922309875 segundos

Resultados para un array de tamaño 50000000 dividido en 2 partes:
Resultado: 25001185.050885387
Tiempo: 45.72059679031372 segundos

Resultados para un array de tamaño 50000000 dividido en 4 partes:
Resultado: 25001185.050884742
Tiempo: 23.355814933776855 segundos



##### b) Usando Numba

In [14]:
from numba import njit, prange

@njit
def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s


X = np.random.rand(value)

print (f"En paralelo usando @njit de numba con {value} valores:\n")


tiempo = %timeit -r 2 -o -q reduc_operation(X) #Una primera ejecución para la compilación inicial

# Utilizando bucle for. 
tiempo = %timeit -r 2 -o -q reduc_operation(X)
print("Tiempo:", tiempo)
print(f"Resultado de la suma en rango [0, {value}]: {reduc_operation(X)}\n")



@njit(parallel=True)
def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in prange(A.size):
        s += A[i]
    return s

value = 5*10**7
X = np.random.rand(value)

print (f"En paralelo usando @njit(parallel=True) y prange de numba con {value} valores:\n")

tiempo = %timeit -r 2 -o -q reduc_operation(X) #Una primera ejecución para la compilación inicial

# Utilizando bucle for. 
tiempo = %timeit -r 2 -o -q reduc_operation(X)
print("Tiempo:", tiempo)
print(f"Resultado de la suma en rango [0, {value}]: {reduc_operation(X)}\n")


En paralelo usando @njit de numba con 50000000 valores:

Tiempo: 203 ms ± 1.86 µs per loop (mean ± std. dev. of 2 runs, 1 loop each)
Resultado de la suma en rango [0, 50000000]: 25000926.615494207

En paralelo usando @njit(parallel=True) y prange de numba con 50000000 valores:

Tiempo: 10.6 ms ± 221 µs per loop (mean ± std. dev. of 2 runs, 100 loops each)
Resultado de la suma en rango [0, 50000000]: 24994847.090746764



Al usar map y Pool de la librería Multiprocessing, se aprecia claramente cómo al duplicar el número de hilos paralelos para a 

La aceleración que proporciona numba sobre la función "reduc_operation" es considerable: reduce los tiempos de ejecución del orden de las decenas de segundo hasta centenas de milisegundo usando @njit. Por otro lado, especificar @njit(parallel=True) aplicando "prange" en la función se acelera hasta los 10ms, llegando a superar en velocidad a las funciones universales de suma utilizadas (de en torno a 70ms).